<a href="https://colab.research.google.com/github/Tanzaniav0825/CS667/blob/main/Project_1_deliverables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install tldextract

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 2.6 MB/s eta 0:00:00


In [3]:
import tldextract
from urllib.parse import urlparse
import re
import random

# Simulated ML Scorer (placeholder)
def simulate_ml_score():
    return round(random.uniform(0.6, 0.95), 2)

# Rule-based credibility checker
def evaluate_credibility(url: str) -> dict:
    explanation_parts = []
    score = 0

    # Extract domain
    ext = tldextract.extract(url)
    domain = f"{ext.domain}.{ext.suffix}"

    # Rule 1: Domain suffix
    if ext.suffix in ["edu", "gov"]:
        score += 0.3
        explanation_parts.append("Government or academic domain.")
    elif ext.suffix in ["org"]:
        score += 0.2
        explanation_parts.append("Non-profit domain.")
    else:
        score += 0.1
        explanation_parts.append("Commercial or unknown domain.")

    # Rule 2: Trusted source list
    trusted_sources = ["nature.com", "sciencedirect.com", "springer.com", "jstor.org", "arxiv.org"]
    if any(t in url for t in trusted_sources):
        score += 0.4
        explanation_parts.append("Domain is a well-known academic publisher.")

    # Rule 3: Publication recency check (placeholder)
    if re.search(r"/20\d{2}/", url):
        score += 0.1
        explanation_parts.append("URL indicates recent publication.")

    # Combine with simulated ML credibility score
    ml_score = simulate_ml_score()
    score += ml_score * 0.2
    explanation_parts.append("ML model suggests high relevance and credibility.")

    final_score = min(round(score, 2), 1.0)
    explanation = " ".join(explanation_parts)

    return {"score": final_score, "explanation": explanation}


In [4]:
# Example URLs to test
test_urls = [
    "https://www.nature.com/articles/s41586-020-03119-7",
    "https://arxiv.org/abs/2401.04566",
    "https://buzzrandomfacts.net/article123.html",
    "https://www.sciencedirect.com/science/article/pii/S0167739X24000430",
    "https://fakehealthnews.biz/cures-you-dont-know"
]

# Run each through the credibility function
for url in test_urls:
    result = evaluate_credibility(url)
    print(f"URL: {url}")
    print(f"Score: {result['score']}")
    print(f"Explanation: {result['explanation']}")
    print("-" * 80)


URL: https://www.nature.com/articles/s41586-020-03119-7
Score: 0.64
Explanation: Commercial or unknown domain. Domain is a well-known academic publisher. ML model suggests high relevance and credibility.
--------------------------------------------------------------------------------
URL: https://arxiv.org/abs/2401.04566
Score: 0.74
Explanation: Non-profit domain. Domain is a well-known academic publisher. ML model suggests high relevance and credibility.
--------------------------------------------------------------------------------
URL: https://buzzrandomfacts.net/article123.html
Score: 0.29
Explanation: Commercial or unknown domain. ML model suggests high relevance and credibility.
--------------------------------------------------------------------------------
URL: https://www.sciencedirect.com/science/article/pii/S0167739X24000430
Score: 0.65
Explanation: Commercial or unknown domain. Domain is a well-known academic publisher. ML model suggests high relevance and credibility.
---